In [1]:
import numpy as np
import pandas as pd

# напишіть функцію гіпотези лінійної регресії у векторному вигляді
def lin_reg(w0, w1, x):
    """
    w0 (scalar) - Перші ваги
    w1 (scalar) - Другі ваги та коефіцієнт для x
    x (scalar) - Ознака
    """
    vector_w = np.array([w0, w1])
    vector_x = np.array([1, x]).reshape(-1, 1)
    return np.dot(vector_w, vector_x)

In [2]:
# створіть функцію для обчислення функції втрат у векторному вигляді
def mse_function(vector_w, X, y):
    """
    vector_w (array) - Вектор параметрів моделі (ваги).
    X (array) - Матриця ознак.
    y (array) - Вектор цільової змінної.
    """
    m = len(y)
    predictions = np.dot(X, vector_w)
    squared_errors = (predictions - y) ** 2
    result = (1 / (2 * m)) * np.sum(squared_errors)
    return result

In [3]:
# реалізуйте один крок градієнтного спуску
def gradient_descent_step(X, y, vector_w, learning_rate):
    """
    X (array) - Матриця ознак (включаючи стовпець з одиницями для зсуву)
    y (array) - Вектор цільових значень
    vector_w (array) - Поточні значення параметрів моделі
    learning_rate (float) - Швидкість навчання (розмір кроку градієнтного спуску)

    return - array: Оновлені значення параметрів моделі
    """
    m = len(y)
    predictions = np.dot(X, vector_w)
    errors = predictions - y
    gradient = (1 / m) * np.dot(X.T, errors)
    vector_w_result = vector_w - learning_rate * gradient
    return vector_w_result

In [4]:
# знайдіть найкращі параметри w для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень
data = pd.read_csv("D:\Python\DataScience homeworks\hw3\Housing.csv")
print(data)

        price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0    13300000  7420         4          2        3      yes        no       no   
1    12250000  8960         4          4        4      yes        no       no   
2    12250000  9960         3          2        2      yes        no      yes   
3    12215000  7500         4          2        2      yes        no      yes   
4    11410000  7420         4          1        2      yes       yes      yes   
..        ...   ...       ...        ...      ...      ...       ...      ...   
540   1820000  3000         2          1        1      yes        no      yes   
541   1767150  2400         3          1        1       no        no       no   
542   1750000  3620         2          1        1      yes        no       no   
543   1750000  2910         3          1        1       no        no       no   
544   1750000  3850         3          1        2      yes        no       no   

    hotwaterheating aircond

In [5]:
X = data[['area', 'bedrooms', 'bathrooms']].values # Беремо необхнідні дані
X = (X - X.mean(axis=0)) / X.std(axis=0) # Нормалізуємо їх
X = np.c_[np.ones(X.shape[0]), X] # Додаємо стовпець з одиницями
y = data['price'].values # Беремо цільові значення
vector_w = np.zeros(X.shape[1]) # Створюємо початкові ваги, array

learning_rate = 0.01
max_iterations = 10000
epsilon = 1e-5
prev_loss = 999999999999

iteration = 0
while iteration < max_iterations:
    vector_w = gradient_descent_step(X, y, vector_w, learning_rate)
    loss = mse_function(vector_w, X, y)
    if abs(prev_loss - loss) < epsilon:
        print(f"Збіжність досягнута на ітерації {iteration}.")
        break
    prev_loss = loss
    iteration += 1        

if iteration == max_iterations:
    print("Попередньо встановлене максимальне число ітерацій досягнуто.")

print("Оптимальні параметри вагів:", vector_w)

Збіжність досягнута на ітерації 2181.
Оптимальні параметри вагів: [4766729.24628011  821214.16789549  299983.75008319  695808.33287788]


In [6]:
# знайдіть ці ж параметри за допомогою аналітичного рішення
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
X = data[['area', 'bedrooms', 'bathrooms']]


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

y = data['price']

model = LinearRegression()
model.fit(X_scaled, y)

vector_w_analitics = [model.intercept_] + model.coef_.tolist()
print("Оптимальні параметри вагів:", vector_w_analitics)

Оптимальні параметри вагів: [4766729.247706422, 821214.1434951875, 299983.57107963285, 695808.5227253672]


In [9]:
# порівняйте отримані результати
print(f"Перші отримані параметри вагів - {vector_w}")
print(f"Другі отримані параметри вагів - {vector_w_analitics}")
print(f"Похибка значення вагів при різних підходах до обчислення - {vector_w - vector_w_analitics}")

Перші отримані параметри вагів - [4766729.24628011  821214.16789549  299983.75008319  695808.33287788]
Другі отримані параметри вагів - [4766729.247706422, 821214.1434951875, 299983.57107963285, 695808.5227253672]
Похибка значення вагів при різних підходах до обчислення - [-0.00142631  0.02440031  0.17900355 -0.18984749]
